In [13]:
import os
import tensorflow as tf
import numpy as np
from keras import backend as K
from keras.applications import vgg16
from keras.layers import Input, merge
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Dense, Dropout, Flatten, Lambda
from keras.models import Sequential, Model
from keras.layers import Lambda, Input, Dense, GlobalAveragePooling2D, Merge, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras import optimizers
from random import shuffle
from scipy.misc import imresize
import itertools
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class ImageNetFeatureExtractor(object):
    def __init__(self, model="vgg16", resize_to=(224, 224)):
        # MODEL_DICT = {"vgg16": VGG16, "vgg19": VGG19, "inception": InceptionV3, "resnet": ResNet50,
        #               "xception": Xception}
        # network = MODEL_DICT[model.lower()]
        self.model_name = model.lower()
        self.model = self.getModel()
        self.preprocess_input = preprocess_input
        self.imageSize = resize_to

    def extract(self, images):
        images = self.preprocess(images)
        return self.model.predict(images)

    def getModel(self):
        model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer("block5_pool").output)

        return intermediate_layer_model

    @property
    def output_shape(self):
        return self.model.compute_output_shape([[None, self.imageSize[0], self.imageSize[1], 3]])

    def resize_images(self, images):
        images = np.array([cv2.resize(image, (self.imageSize[0], self.imageSize[1])) for image in images])
        return images

    def preprocess(self, images):
        images = self.resize_images(images)
        images = self.preprocess_input(images.astype("float"))
        return images

def concat_tensors(tensors, axis=-1):
    return K.concatenate([K.expand_dims(t, axis=axis) for t in tensors])


def get_small_network(input_shape=(None, 4, 4, 512)):
    model = Sequential()
    model.add(GlobalAveragePooling2D(input_shape=input_shape[1:]))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation="relu"))
    #model.add(Dense(128, activation="relu"))
    return model

def get_triplet_network(input_shape=(None, 4, 4, 512)):
    base_model = get_small_network(input_shape=input_shape)

    anchor_input = Input(input_shape[1:])
    positive_input = Input(input_shape[1:])
    negative_input = Input(input_shape[1:])

    anchor_embeddings = base_model(anchor_input)
    positive_embeddings = base_model(positive_input)
    negative_embeddings = base_model(negative_input)

    output = Lambda(concat_tensors)([anchor_embeddings, positive_embeddings, negative_embeddings])
    model = Model([anchor_input, positive_input, negative_input], output)

    return model

In [4]:
def get_triplets(data, labels):
    pos_label, neg_label = np.random.choice(labels, 2, replace=False)
    pos_indexes = np.where(labels == pos_label)[0]
    neg_indexes = np.where(labels == neg_label)[0]
    np.random.shuffle(pos_indexes)
    np.random.shuffle(neg_indexes)
    anchor = data[pos_indexes[0]]
    positive = data[pos_indexes[-1]]
    negative = data[neg_indexes[0]]
    return (anchor, positive, negative)

In [5]:
def get_hard_neg(anchor, embeddings, mask):
    embeddings_masked = embeddings * mask
    distances = tf.norm(anchor-embeddings_masked)
    dist_idx  = tf.argmin(distances)
    return dist_idx

In [6]:
def batch_hard_negative_triplet_loss(labels, embeddings):

    margin = 1
    feat_size = 512
    num_identities = 6
    samples_per_id = 20

    triplet_list = []
    pos_dist_list, neg_dist_list = [], []

    for i in range(num_identities * samples_per_id):
        anchor = embeddings[i]
        mask = tf.logical_not(tf.equal(labels[i], labels))
        mask = tf.expand_dims(mask, 1)
        anchor_tiled = tf.tile([anchor], [num_identities * samples_per_id, 1])
        hard_neg = get_hard_neg(anchor_tiled, embeddings, mask)
        mult = (i // samples_per_id)+1 
        for j in range(i+1,samples_per_id*mult):
            triplet_list += [(anchor, embeddings[j], embeddings[hard_neg])]

    triplets = tf.stack(triplet_list)
    
    for i in range(num_identities * (samples_per_id * (samples_per_id-1)) // 2):
        pos_dist_list += [tf.norm(triplets[i,0] - triplets[i,1])]
        neg_dist_list += [tf.norm(triplets[i,0] - triplets[i,2])]

    pos_dist = tf.stack(pos_dist_list)
    neg_dist = tf.stack(neg_dist_list)

    triplet_loss = tf.reduce_mean(tf.maximum(pos_dist - neg_dist + margin, 0.))

    return triplet_loss

In [ ]:
def create_triples(image_dir):
    img_groups = {}
    for img_file in os.listdir(image_dir):
        prefix, suffix = img_file.split(".")
        gid, pid = prefix[0:4], prefix[4:]
        if img_groups.has_key(gid):
            img_groups[gid].append(pid)
        else:
            img_groups[gid] = [pid]
    pos_triples, neg_triples = [], []
    # positive pairs are any combination of images in same group
    for key in img_groups.keys():
        triples = [(key + x[0] + ".jpg", key + x[1] + ".jpg", 1) 
                 for x in itertools.combinations(img_groups[key], 2)]
        pos_triples.extend(triples)
    # need equal number of negative examples
    group_names = list(img_groups.keys())
    for i in range(len(pos_triples)):
        g1, g2 = np.random.choice(np.arange(len(group_names)), size=2, replace=False)
        left = get_random_image(img_groups, group_names, g1)
        right = get_random_image(img_groups, group_names, g2)
        neg_triples.append((left, right, 0))
    pos_triples.extend(neg_triples)
    shuffle(pos_triples)
    return pos_triples

In [8]:
def extract_features(hdf5_path):
    db = h5py.File(hdf5_path,mode="r")
    features = db["features"][:]
    labels = db["labels"][:]

    return (features, labels)

def extract_embeddings(features, model):
    embeddings = model.predict([features, features, features])
    return embeddings[:,:,0]

In [15]:
model_check_point_loc = 'C:/Users/hp/Downloads/data/similarity/vgg16_cats_dogs_sm.h5'
features, labels = extract_features('C:/Users/hp/Downloads/data/similarity/similarity_db.hdf5')

model = get_triplet_network(features.shape)

#model.compile(optimizer=optimizers.Adam(lr=1e-4), loss=batch_hard_negative_triplet_loss, metrics=[])

In [16]:
features.shape

(1200, 4, 4, 512)